In [1]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [2]:
df = pd.read_csv(f"output/features_labeled.csv")

In [3]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '23_106', 'score': 0.5400894187779...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)']
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)']
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s..."
3,2022-08-22 04:04,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s..."
4,2022-08-22 04:11,frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '104_106', 'score': 0.605582524271...",['CpuUsageRate(%)']


In [4]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))
df['root_cause'] = df['root_cause'].apply(lambda x: ast.literal_eval(x))

In [5]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [6]:
df['root_cause'].apply(str).unique()

array(["['CpuUsageRate(%)']",
       "['Serving product page started', 'GetProduct start']",
       "['Placing order started', 'Order placed complete']",
       "['NetworkP90(ms)']",
       "['Start charge card', 'Charge successfully']",
       "['Received ad request', 'No context provided']",
       "['Query product with name', 'Query product successfully']"],
      dtype=object)

In [7]:
def get_timediff(inject_time, abnormal_time):
    inject_dt = datetime.strptime(inject_time, '%Y-%m-%d %H:%M:%S')
    abnormal_dt = datetime.strptime(abnormal_time, '%Y-%m-%d %H:%M')
    
    diff = max(1, abs(inject_dt.minute - abnormal_dt.minute))
    return diff

df["inject_comp_tdiff"] = df.apply(lambda row: get_timediff(row['inject_time'], row['abnormal_time']), axis=1)

In [8]:
df.inject_comp_tdiff.value_counts()

1    56
2    56
Name: inject_comp_tdiff, dtype: int64

In [9]:
log_template_miner = get_miner()

In [10]:
def get_nezha_rank(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
            
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
            else:
                if "events_actual" in result_list[i].keys() and str(root_cause[0]) in from_id_to_template(int(result_list[i]["events_actual"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
            rank += 1
            
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
            rank += 1
    
    if not rc_found_flag:
        return -1
    return rank

In [11]:
df["nezha_rank_050_correct"] = df.apply(lambda row: get_nezha_rank(row['result_list_inhealthy_moreoften'], row['root_cause'], row['inject_pod']), axis=1)

In [12]:
def get_nezha_rank_orig(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break   
            else:
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank = rank + 1
                elif i == 0:
                    rank = rank + 1
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break  
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank = rank + 1
                elif i == 0:
                    rank = rank + 1
    if not rc_found_flag:
        return -1
    return rank

In [13]:
df["nezha_rank_050_orig"] = df.apply(lambda row: get_nezha_rank_orig(row['result_list_inhealthy_moreoften'], row['root_cause'], row['inject_pod']), axis=1)

In [15]:
df[["nezha_rank_050_orig", "nezha_rank_050_correct"]]

,nezha_rank_050_orig,nezha_rank_050_correct
0,1,1
1,1,1
2,2,2
3,2,2
4,4,5
...,...,...
107,-1,-1
108,1,1
109,1,1
110,1,1


In [20]:
# ais@1 on orig nezha, in nezha log: 0.8928571428571429
mask = (df.inject_comp_tdiff == 2) & (df.nezha_rank_050_orig == 1)
len(df[mask]) / 56

0.8928571428571429

In [21]:
# ais@1 on corrected nezha
mask = (df.inject_comp_tdiff == 2) & (df.nezha_rank_050_correct == 1)
len(df[mask]) / 56

0.8928571428571429

In [22]:
# ais@1 on data after 1 min on orig nezha
mask = (df.inject_comp_tdiff == 1) & (df.nezha_rank_050_orig == 1)
len(df[mask]) / 56

0.8214285714285714

In [23]:
# ais@1 on data after 1 min on corrected nezha
mask = (df.inject_comp_tdiff == 1) & (df.nezha_rank_050_correct == 1)
len(df[mask]) / 56

0.8035714285714286

In [24]:
df.columns

Index(['abnormal_time', 'inject_pod', 'inject_time', 'inject_type',
       'normal_time', 'result_list_inhealthy_lessoften',
       'result_list_inhealthy_moreoften', 'root_cause', 'inject_comp_tdiff',
       'nezha_rank_050_correct', 'nezha_rank_050_orig'],
      dtype='object')

In [25]:
df.to_csv(f"output/features_labeled_4llm.csv", index=False)